In [1]:
import pandas as pd
import numpy as np

# Data Wrangling Imports
from datetime import datetime

# Data Analysis Imports
import statistics
import seaborn as sns
from scipy import linalg
from matplotlib import style
from matplotlib import ticker
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

# Data Visualization Imports
#import cufflinks as cf
#import chart_studio.plotly as py
#import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.gridspec import GridSpec

# Econometry Imports
import pandas_ta as ta

# Other Imports
import os
from itertools import product
import warnings
import talib
warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:.10f}".format

### 2- Depois de cadastrar uma conta na [Binance](https://www.binance.com/pt-BR) é necessário inserir os parâmetros key e secret da API

In [2]:
from binance.client import Client
%run chaves02.ipynb
client = Client(key,secret)

In [3]:
# Escolhe o par
TRADING_PAIR = 'BTCBRL'
# Data de Inicio da Captura dos Dados
STARTING_DATE = "5 Mar, 2021"
# Data Final da Captura dos Dados
END_DATE = "12 Mar, 2021"

In [4]:
def process_message(msg):
    if msg['e'] == 'error':
        bm.stop_socket(conn_key)
        bm.close()
    else:
        pass

In [5]:
time_res = client.get_server_time()
time_res

{'serverTime': 1617506797285}

### 3- Obtenção de informações via API da Binance e construção do dataframe __data__

### [Binance Websockets API](https://python-binance.readthedocs.io/en/latest/websockets.html)

In [6]:
# Importing Binance Websockets API
from binance.client import *
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor

# Initialize Binance Socket Manager
bm = BinanceSocketManager(client, user_timeout=60)
list_messages = []
conn_key = bm.start_trade_socket(TRADING_PAIR, process_message)
bm.start


<bound method Thread.start of <BinanceSocketManager(Thread-6, initial)>>

In [7]:
candles = client.get_klines(symbol=TRADING_PAIR, interval=Client.KLINE_INTERVAL_1MINUTE)

In [8]:
# Old Get klines from Binance Python API
#klines = client.get_historical_klines("ONEBNB", Client.KLINE_INTERVAL_1MINUTE, STARTING_DATE, END_DATE)

In [9]:
indexes = ['Open Time', 'Open', 'High','Low', 'Close', 'Volume', 'Close Time', 'QAV', 'No. Trades', 'Taker BBAV', 'Taker BQAV', 'Ignore']
data = pd.DataFrame(columns=indexes,data=candles)

In [10]:
len(data)

500

In [11]:
data.to_csv("CryptoBot_202103006Mv001.csv", index = False)

In [12]:
data = pd.read_csv("CryptoBot_202103006Mv001.csv")

In [13]:
data.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,QAV,No. Trades,Taker BBAV,Taker BQAV,Ignore
0,1617476820000,338755.0000000000,339000.0000000000,338755.0000000000,339000.0000000000,0.0556840000,1617476879999,18868.9730060000,29,0.0486270000,16478.3335330000,0
1,1617476880000,339000.0000000000,339850.0000000000,339000.0000000000,339546.0000000000,0.0464710000,1617476939999,15777.6100720000,40,0.0266150000,9039.2937230000,0
2,1617476940000,339348.0000000000,339844.0000000000,339348.0000000000,339546.0000000000,0.0066590000,1617476999999,2261.2176340000,15,0.0017940000,609.5430170000,0
3,1617477000000,339366.0000000000,339842.0000000000,339191.0000000000,339546.0000000000,0.0048520000,1617477059999,1646.8587910000,22,0.0008650000,293.8803050000,0
4,1617477060000,339740.0000000000,339740.0000000000,339150.0000000000,339150.0000000000,0.0162030000,1617477119999,5501.8775480000,16,0.0155200000,5270.2021000000,0


In [14]:
data.dtypes
data['Open Time'] = data['Open Time'].apply(lambda x: datetime.fromtimestamp(int(x)/1000))
data.set_index(['Open Time'], inplace=True)
# data['Open'] = data['Open'].astype(float)
# data['High'] = data['High'].astype(float)
# data['Low'] = data['Low'].astype(float)
data['Close'] = data['Close'].astype(float)
data['Volume'] = data['Volume'].astype(float)
# data['QAV'] = data['QAV'].astype(float)
# data['Taker BBAV'] = data['Taker BBAV'].astype(float)
# data['Taker BQAV'] = data['Taker BQAV'].astype(float)
open_ = data['Open']
high = data['High']
low = data['Low']
close = data['Close']
volume = data['Volume']